##  $$\color{blue}{\text{2.131: Artificial Muscle}} $$

#### <center> HH, 9 May 2023

Version Notes:  
-  9 May 2023 Started from StepMotor code, added Potentiometer and Load Cell codes

***

#### Objectives

Notes from Michael:
-map displacement to load/vice versa, calculate gain/phase/transfer function for dif freq.s (can try to fit second order model to transfer function)
-try 100 freq.s (call digilent with SDK python examples)
-define in terms of mechanical impedance vs freq.
-any sense of what mech impedance to expect for 

-drive a geared step motor (see https://www.omc-stepperonline.com/download/17HS13-0404S-PG5.pdf ) to desired position or until desired force condition

-input waveform to LFA

-read position from potentiometer

-read load from load cell

stepper is driven by a TinkerForge step motor controller (called the TinkerForge Silent Stepper Bricklet (see https://www.tinkerforge.com/en/doc/Hardware/Bricklets/Silent_Stepper_V2.html ) or the TinkerForge Silent Stepper Brick (see https://www.tinkerforge.com/en/doc/Hardware/Bricks/Silent_Stepper_Brick.html ) and powered by DeWalt 20V battery.
The step motor needs to rotate by 5.18 turns to produce a single turn of the gear box output shaft (i.e., its gear ratio is 5.18:1)

Waveform is delivered from Digilent to amplifier to LFA.
Potentiometer is read by bricklet (Industrial Dual Analog In).
Load cell is read by load cell bricklet.

#### Import Standard Libraries

In [90]:
import numpy as np # works with complex but one of the arguments must be complex
import matplotlib.pyplot as plt
import time
from scipy import stats
%matplotlib inline
#%config InlineBackend.figure_format = 'png' # use 'png' instead of 'svg' when converting to HTML for assignment submission
%config InlineBackend.figure_format = 'svg' # this is key to improving plot resolution

#### Setup Devices

In [91]:
# Only need to do this after a JupyterLab upgrade
#!pip install tinkerforge

In [92]:
HOST = "localhost"
PORT = 4223
UID_SS = "21bG" # Change XYZ to the UID of your Silent Stepper Bricklet
UID = "LZv" # Change XYZ to the UID of your Industrial Dual Analog In Bricklet 2.0
UID_LC = "Kin" # Change XYZ to the UID of your Load Cell Bricklet 2.0

from tinkerforge.ip_connection import IPConnection
from tinkerforge.bricklet_silent_stepper_v2 import BrickletSilentStepperV2
#from tinkerforge.brick_silent_stepper import BrickSilentStepper
from tinkerforge.bricklet_imu_v3 import BrickletIMUV3
from tinkerforge.bricklet_thermocouple_v2 import BrickletThermocoupleV2

from tinkerforge.bricklet_industrial_dual_analog_in_v2 import BrickletIndustrialDualAnalogInV2

from tinkerforge.bricklet_load_cell_v2 import BrickletLoadCellV2

ipcon = IPConnection() # Create IP connection
# remember to use the TinkerForge viewer to get the UID's of the various devices below (red string)

ss = BrickletSilentStepperV2(UID_SS, ipcon) # Create device object
#ss = BrickSilentStepper(UID, ipcon) # Create device object
idai = BrickletIndustrialDualAnalogInV2(UID, ipcon) # Create device object
lc = BrickletLoadCellV2(UID_LC, ipcon) # Create device object

ipcon.connect(HOST, PORT) # Connect to brickd
time.sleep(1) # Don't use device before ipcon is connected

#### Set TinkerForge Parameters

In [93]:
# Step motors have 200 marco steps/rev
# the gear box is 5.18:1 which means that the step motor has to rotate 5.18 revs to produce 1 rev of gear box output shaft
ss.set_motor_current(800) # 800 mA
ss.set_step_configuration(ss.STEP_RESOLUTION_128, True) # 128 micro-steps (usteps) per macro step
# 128 * 200 usteps per rev = 25,600 usteps/rev
ss.set_max_velocity(round(1280*5.18)) # Velocity 1,280 * 5.18 usteps/s = 0.05 rev /s of gear box output shaft
# max velocity is 2**16-1 = 65,535
ss.set_speed_ramping(65535,65535) # max accel and decel

ss.set_enabled(False) # Disable motor power
#ss.enable() # Enable motor power

#### Run Experiment

In [ ]:
LC_file = datetime.now().strftime('DMA-%Y-%m-%d-%H-%M-%S.csv')
Pot_file = datetime.now().strftime('DMA-%Y-%m-%d-%H-%M-%S.csv')

# Callback function for weight callback
def cb_weight(weight):

    with open(LC_file, 'a') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow([datetime.now().strftime('%H:%M:%S:%f'),weight])
        f_object.close()
        #print("Weight: " + str(weight) + " g")
        
# Callback function for voltage callback
def cb_voltage(channel, voltage):
    print("Channel: " + str(channel))
    print("Voltage: " + str(voltage/1000.0) + " V")
    print("")
        
# Set period for weight callback to 1s (1000ms) without a threshold
lc.set_weight_callback_configuration(12, False, "x", 0, 0)
    
# Callback
lc.register_callback(lc.CALLBACK_WEIGHT, cb_weight)
    
input("Press key to exit\n") # Use raw_input() in Python 2
ipcon.disconnect()

#### Plot Data

In [1]:
plt.rcParams['figure.figsize'] = [8, 5]
fig, ax = plt.subplots()
#plt.plot(t,heading,'ro',markersize=1)
plt.xlabel('Time (s)')
#plt.ylabel('Force ()')
plt.ylabel('Displacement ()')
plt.title('')
ax.grid()
plt.show()

NameError: name 'plt' is not defined